In [22]:
import json
import requests
import pandas as pd
import openai
from pathlib import Path
from dotenv import load_dotenv
import os

load_dotenv()

# Initialize new OpenAI client interface
client = openai.OpenAI()

# Configuration
WEBHOOK_URL     = "https://congliu.app.n8n.cloud/webhook/413ab7a1-30ae-4276-a5d0-8d5ecda4f7a3"
BENCHMARK_FILE  = Path("frontdesk_agent_eval_benchmark.jsonl")
BEST_PROMPT_FILE = Path("best_prompt.txt")

# Helpers
def call_agent(query: str, system_prompt: str, webhook_url: str = WEBHOOK_URL) -> dict:
    payload = {
        "sessionId":      "benchmark",
        "message":        query,
        "from_agent":     "patient_agent",
        "system_prompt":  system_prompt,
        "fhir_server_url":""
    }
    resp = requests.post(webhook_url, json=payload, timeout=(5,30))
    resp.raise_for_status()
    return resp.json()

def score_routing(reply: dict, gold: dict) -> float:
    s = 0
    s += int(reply.get("to_agent") == gold["to_agent"])
    s += int(reply.get("end_conversation") == gold["end_conversation"])
    return s / 2.0

def score_llm(reply: dict, gold: dict, ideal_content: str, ideal_token: str) -> float:
    prompt = {
        "system": (
            "You are evaluating a telemedicine front-desk agent’s response. "
            "The agent should emit the correct routing token and provide content matching the ideal."
        ),
        "user": (
            f"Gold routing: to_agent={gold['to_agent']}, end_conversation={gold['end_conversation']}\n"
            f"Ideal content: {ideal_content}\n"
            f"Ideal token: {ideal_token}\n\n"
            "Here is the JSON reply from the agent:\n"
            f"{json.dumps(reply, indent=2)}\n\n"
            "Rate from 0.0 to 1.0 how well this reply matches routing and content. "
            "Respond with exactly a JSON object (no markdown fences, no extra text): {\"score\": <float> }"
        )
    }
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role":"system","content": prompt["system"]},
            {"role":"user","content":   prompt["user"]},
        ],
        temperature=0.0,
        max_tokens=16
    )
    text = resp.choices[0].message.content.strip()
    try:
        return json.loads(text)["score"]
    except:
        print("Scoring failure")
        return 0.0
    

# Load cases and prompt
cases       = [json.loads(line) for line in BENCHMARK_FILE.open() if line.strip()]




In [23]:
# Load optimized prompt
#base_prompt = BEST_PROMPT_FILE.read_text().strip()

# USING THE BASE PROMPT FOR NOW
base_prompt = """
You are a front desk agent for a telemedicine clinic. In every interaction you must check the Redis memory for context.

If the patient’s question is strictly administrative (e.g., about billing, clinic policies, etc.), or if the issue is related to insurance (e.g., coverage, copays, claims), forward the entire message verbatim to the administrative agent. End your output with the token "<ADMIN>".

If the patient’s question requires a medical diagnosis or specialized medical knowledge, forward the entire message verbatim to the education agent. End your output with the token "<EDUCATION>".

If the patient is specifically requesting to book, reschedule, or cancel an appointment, or asking about available time slots, clinic hours, or scheduling logistics, you must forward the entire message verbatim to the scheduling agent. Do not attempt to process scheduling requests yourself. End your output with the token <SCHEDULING>.

By default, you must always end your reply with the token <PATIENT>, unless it is a referral to an agent.

If at any point the conversation should conclude, output the single token: "<STOP>".
""".strip()


base_prompt_textgrad_enhanced = """
You are a front desk agent for a telemedicine clinic. In every interaction you must check the Redis memory for context.

If the patient’s question is strictly administrative (e.g., about billing, clinic policies, etc.), or if the issue is related to insurance (e.g., coverage, copays, claims), forward the entire message verbatim to the administrative agent. End your output with the token "<ADMIN>".

If the patient’s question requires a medical diagnosis or specialized medical knowledge, forward the entire message verbatim to the education agent. End your output with the token "<EDUCATION>".

If the patient is specifically requesting to book, reschedule, or cancel an appointment, or asking about available time slots, clinic hours, or scheduling logistics, you must forward the entire message verbatim to the scheduling agent. Do not attempt to process scheduling requests yourself. End your output with the token <SCHEDULING>.

By default, you must always end your reply with the token <PATIENT>, unless it is a referral to an agent.

If at any point the conversation should conclude, output the single token: "<STOP>".



Always include the correct routing token (<ADMIN>, <EDUCATION>, <SCHEDULING>, <PATIENT>, or <STOP>) at the end of your raw reply.

Acknowledge any user request (e.g. “I’ve updated that for you,” “Got it, canceling now,” “Rescheduling to 3 PM”) before forwarding to an agent.

When a query is ambiguous, ask a single, clear follow-up question (e.g. “Which test results are you referring to?” or “Which specialist do you need?”).

In safety or emergency cases, immediately advise the user to call 911 or a crisis hotline before routing to the education agent.

Offer practical clinic-logistics info when safe (parking, wifi, languages spoken, service-dog policy) without forwarding.

Maintain a friendly, empathetic tone. You may sprinkle very light, inoffensive humor for chit-chat but never let it override clarity.

Always keep explanations concise, direct and strictly relevant to the user’s question.


""".strip()

In [26]:
# Run benchmark
records = []
total = len(cases)

for idx, ex in enumerate(cases, start=1):
    # Unpack
    query         = ex["query"]
    gold          = ex["gold"]
    ideal_content = ex["ideal_content"]
    ideal_token   = ex["ideal_token"]
    category      = ex.get("task_category", "")

    # 1) Log what we’re doing
    print(f"\n[{idx}/{total}] Category: {category}")
    print(" Query:", query)

    # 2) Call the agent CHANGE PROMPT
    reply = call_agent(query, base_prompt)
    print(" Reply JSON:", reply)

    # 3) Deterministic routing score
    det_score = score_routing(reply, gold)
    print(f" Deterministic score: {det_score:.2f}")

    # 4) LLM-based content+routing score
    llm_score = score_llm(reply, gold, ideal_content, ideal_token)
    print(f" LLM score:          {llm_score}")

    # 5) Store
    records.append({
        "task_category":      category,
        "query":              query,
        "to_agent":           reply.get("to_agent"),
        "end_conversation":   reply.get("end_conversation"),
        "execution_id":       reply.get("execution_id"),
        "deterministic_score": det_score,
        "llm_score":          llm_score
    })


[1/42] Category: straight_admin
 Query: Please change my email address on file.
 Reply JSON: {'output': 'Please change my email address on file.', 'execution_id': '3849', 'to_agent': 'administrative_agent', 'from_agent': 'frontdesk_agent', 'end_conversation': False}
 Deterministic score: 1.00
 LLM score:          0.5

[2/42] Category: straight_admin
 Query: I’d like to request a payment plan for my outstanding balance.
 Reply JSON: {'output': 'I’d like to request a payment plan for my outstanding balance.', 'execution_id': '3850', 'to_agent': 'administrative_agent', 'from_agent': 'frontdesk_agent', 'end_conversation': False}
 Deterministic score: 1.00
 LLM score:          0.5

[3/42] Category: straight_admin
 Query: Can you send me a copy of my insurance coverage details?
 Reply JSON: {'output': 'Can you send me a copy of my insurance coverage details?', 'execution_id': '3851', 'to_agent': 'administrative_agent', 'from_agent': 'frontdesk_agent', 'end_conversation': False}
 Determinist

In [27]:
import pandas as pd

# 1) Turn records into a DataFrame
df = pd.DataFrame(records)

# 2) Add an 'average_score' column
df['average_score'] = (df['deterministic_score'] + df['llm_score']) / 2

# 3) Print the full table of results
print("=== Full Benchmark Results ===")
print(df.to_string(index=False))

# 4) Compute and print average scores per task_category
summary = df.groupby('task_category')['average_score'].mean().reset_index()
print("\n=== Average Score by Category ===")
print(summary.to_string(index=False))

# 5) Compute and print overall average across all test cases
overall_avg = df['average_score'].mean()
print(f"\n=== Overall Average Score Across All Cases: {overall_avg:.3f} ===")


=== Full Benchmark Results ===
        task_category                                                             query             to_agent  end_conversation execution_id  deterministic_score  llm_score  average_score
       straight_admin                           Please change my email address on file. administrative_agent             False         3849                  1.0        0.5           0.75
       straight_admin    I’d like to request a payment plan for my outstanding balance. administrative_agent             False         3850                  1.0        0.5           0.75
       straight_admin          Can you send me a copy of my insurance coverage details? administrative_agent             False         3851                  1.0        0.0           0.50
       straight_admin                  What documents do you need to update my address? administrative_agent             False         3852                  1.0        0.5           0.75
       straight_admin           Pl

In [28]:
#df_results_4o_base_prompt = df
#df_results_4o_textgrad_prompt = df
df_results_o1_base_prompt = df


In [16]:
df_results_4o_base_prompt

,task_category,query,to_agent,end_conversation,execution_id,deterministic_score,llm_score,average_score
0,straight_admin,Please change my email address on file.,administrative_agent,False,3676,1.0,0.5,0.75
1,straight_admin,I’d like to request a payment plan for my outs...,administrative_agent,False,3677,1.0,0.5,0.75
2,straight_admin,Can you send me a copy of my insurance coverag...,administrative_agent,False,3678,1.0,0.0,0.50
3,straight_admin,What documents do you need to update my address?,administrative_agent,False,3679,1.0,0.5,0.75
4,straight_admin,Please provide a remittance advice for my last...,administrative_agent,False,3680,1.0,0.5,0.75
5,straight_education,What does the HbA1c test measure?,education_agent,False,3681,1.0,0.5,0.75
6,straight_education,What is a normal resting heart rate?,patient_agent,False,3682,0.5,0.5,0.50
7,straight_education,What are the symptoms of type 2 diabetes?,education_agent,False,3683,1.0,0.5,0.75
8,straight_education,How do vaccines work to protect us?,education_agent,False,3684,1.0,0.5,0.75
9,straight_education,What causes antibiotic resistance?,education_agent,False,3685,1.0,0.5,0.75


In [17]:
df_results_4o_textgrad_prompt

,task_category,query,to_agent,end_conversation,execution_id,deterministic_score,llm_score,average_score
0,straight_admin,Please change my email address on file.,patient_agent,False,3718,0.5,0.0,0.25
1,straight_admin,I’d like to request a payment plan for my outs...,administrative_agent,False,3719,1.0,1.0,1.00
2,straight_admin,Can you send me a copy of my insurance coverag...,administrative_agent,False,3720,1.0,0.8,0.90
3,straight_admin,What documents do you need to update my address?,patient_agent,False,3721,0.5,0.0,0.25
4,straight_admin,Please provide a remittance advice for my last...,administrative_agent,False,3722,1.0,1.0,1.00
5,straight_education,What does the HbA1c test measure?,education_agent,False,3723,1.0,1.0,1.00
6,straight_education,What is a normal resting heart rate?,patient_agent,False,3724,0.5,0.5,0.50
7,straight_education,What are the symptoms of type 2 diabetes?,education_agent,False,3725,1.0,0.5,0.75
8,straight_education,How do vaccines work to protect us?,education_agent,False,3726,1.0,1.0,1.00
9,straight_education,What causes antibiotic resistance?,education_agent,False,3727,1.0,1.0,1.00


In [29]:
df_results_o1_base_prompt

,task_category,query,to_agent,end_conversation,execution_id,deterministic_score,llm_score,average_score
0,straight_admin,Please change my email address on file.,administrative_agent,False,3849,1.0,0.5,0.75
1,straight_admin,I’d like to request a payment plan for my outs...,administrative_agent,False,3850,1.0,0.5,0.75
2,straight_admin,Can you send me a copy of my insurance coverag...,administrative_agent,False,3851,1.0,0.0,0.50
3,straight_admin,What documents do you need to update my address?,administrative_agent,False,3852,1.0,0.5,0.75
4,straight_admin,Please provide a remittance advice for my last...,administrative_agent,False,3853,1.0,0.5,0.75
5,straight_education,What does the HbA1c test measure?,education_agent,False,3854,1.0,0.5,0.75
6,straight_education,What is a normal resting heart rate?,education_agent,False,3855,1.0,0.5,0.75
7,straight_education,What are the symptoms of type 2 diabetes?,education_agent,False,3856,1.0,0.5,0.75
8,straight_education,How do vaccines work to protect us?,education_agent,False,3857,1.0,0.5,0.75
9,straight_education,What causes antibiotic resistance?,education_agent,False,3858,1.0,0.5,0.75


In [31]:
# Combining the reuslts

# 1. Add a 'model' column to each dataframe
df_results_4o_base_prompt["model"]        = "4o-base"
df_results_4o_textgrad_prompt["model"]    = "4o-textgrad"
df_results_o1_base_prompt["model"]        = "o1-base"

# 2. Combine them into one
df_all = pd.concat([
    df_results_4o_base_prompt,
    df_results_4o_textgrad_prompt,
    df_results_o1_base_prompt
], ignore_index=True)

# Optional: verify
print(df_all["model"].value_counts())


model
4o-base        42
4o-textgrad    42
o1-base        42
Name: count, dtype: int64


In [33]:
df_all_copy = df_all

In [34]:
# 1. Import the function
from fetch_and_parse_n8n_execution_log import fetch_and_parse_n8n_execution_log

# 2. Define your n8n webhook base URL (this must return execution logs)
N8N_LOG_WEBHOOK = "https://congliu.app.n8n.cloud/webhook/9e17d9af-78a5-46df-bb0f-76376c1eba3e"

# 3. Prepare empty columns
df_all["token_total"] = None
df_all["token_input"] = None
df_all["token_output"] = None

# 4. Loop through each row to populate the token metrics
for i, row in df_all.iterrows():
    exec_id = row.get("execution_id")
    try:
        log_data = fetch_and_parse_n8n_execution_log(exec_id, N8N_LOG_WEBHOOK)

        df_all.at[i, "token_total"]  = log_data.get("token_total")
        df_all.at[i, "token_input"]  = log_data.get("token_input")
        df_all.at[i, "token_output"] = log_data.get("token_output")

        print(f"[{i+1}/{len(df_all)}] Parsed execution ID {exec_id} ✓")

    except Exception as e:
        print(f"[{i+1}/{len(df_all)}] Failed for execution ID {exec_id}: {e}")


[1/126] Parsed execution ID 3676 ✓
[2/126] Parsed execution ID 3677 ✓
[3/126] Parsed execution ID 3678 ✓
[4/126] Parsed execution ID 3679 ✓
[5/126] Parsed execution ID 3680 ✓
[6/126] Parsed execution ID 3681 ✓
[7/126] Parsed execution ID 3682 ✓
[8/126] Parsed execution ID 3683 ✓
[9/126] Parsed execution ID 3684 ✓
[10/126] Parsed execution ID 3685 ✓
[11/126] Parsed execution ID 3686 ✓
[12/126] Parsed execution ID 3687 ✓
[13/126] Parsed execution ID 3688 ✓
[14/126] Parsed execution ID 3689 ✓
[15/126] Parsed execution ID 3690 ✓
[16/126] Parsed execution ID 3691 ✓
[17/126] Parsed execution ID 3692 ✓
[18/126] Parsed execution ID 3693 ✓
[19/126] Parsed execution ID 3694 ✓
[20/126] Parsed execution ID 3695 ✓
[21/126] Parsed execution ID 3696 ✓
[22/126] Parsed execution ID 3697 ✓
[23/126] Parsed execution ID 3698 ✓
[24/126] Parsed execution ID 3699 ✓
[25/126] Parsed execution ID 3700 ✓
[26/126] Parsed execution ID 3701 ✓
[27/126] Parsed execution ID 3702 ✓
[28/126] Parsed execution ID 3703 ✓
[

In [42]:
# Updated per‐token prices (USD per token)
PRICE_TABLE = {
    "4o": {"in": 2.50  / 1_000_000,  "out": 10.00 / 1_000_000},
    "o1": {"in":15.00  / 1_000_000,  "out": 60.00 / 1_000_000},
}

In [43]:
def get_token_prices(model_name):
    key = model_name.split("-",1)[0]  # "4o" or "o1"
    return PRICE_TABLE.get(key)

def compute_cost(row):
    prices = get_token_prices(row["model"])
    return (
        row.get("token_input")  * prices["in"]
      + row.get("token_output") * prices["out"]
    )

df_all["token_cost"] = df_all.apply(compute_cost, axis=1).round(8)

print(df_all[["model","token_input","token_output","token_cost"]].head())
print(f"\nTotal cost: ${df_all['token_cost'].sum():.4f}")


     model token_input token_output  token_cost
0  4o-base         249           25    0.000873
1  4o-base         254           16    0.000795
2  4o-base         253           15    0.000782
3  4o-base         251           13    0.000758
4  4o-base         252           14    0.000770

Total cost: $1.1754


In [45]:
print(df_all.columns.tolist())

['task_category', 'query', 'to_agent', 'end_conversation', 'execution_id', 'deterministic_score', 'llm_score', 'average_score', 'model', 'token_total', 'token_input', 'token_output', 'token_cost']


In [49]:
score_by_category_and_model = df_all.pivot_table(
    index="task_category",
    columns="model",
    values="average_score",
    aggfunc="mean"
).round(4)

print("=== Mean Average Score per Task Category per Model ===")
print(score_by_category_and_model)

overall_avg_by_model = df_all.groupby("model")["average_score"].mean().round(4)

print("\n=== Overall Average Score Across All Cases ===")
print(overall_avg_by_model)

summary_tokens_cost = df_all.groupby("model").agg({
    "token_total": "sum",
    "token_cost": "sum"
}).round(2)

print("\n=== Total Tokens and Cost per Model ===")
print(summary_tokens_cost)

=== Mean Average Score per Task Category per Model ===
model                  4o-base  4o-textgrad  o1-base
task_category                                       
chitchat/out_of_scope    0.750        0.860     0.75
clarification            0.650        0.640     0.75
conversation_end         0.825        0.875     0.75
multi_intent             0.450        0.750     0.82
safety                   0.770        0.730     0.65
straight_admin           0.700        0.680     0.70
straight_education       0.700        0.850     0.75
straight_patient         0.490        0.900     0.25
straight_scheduling      0.750        0.850     0.75

=== Overall Average Score Across All Cases ===
model
4o-base        0.6655
4o-textgrad    0.7869
o1-base        0.6810
Name: average_score, dtype: float64

=== Total Tokens and Cost per Model ===
            token_total  token_cost
model                              
4o-base           11619        0.04
4o-textgrad       20752        0.06
o1-base           257

In [50]:
# 1. Per-category score
score_table = df_all.pivot_table(
    index="task_category",
    columns="model",
    values="average_score",
    aggfunc="mean"
).round(4)

# 2. Overall score
overall_avg = df_all.groupby("model")["average_score"].mean().to_frame().T.round(4)
overall_avg.index = ["Overall Average Score"]

# 3. Token + cost summary
token_cost_summary = df_all.groupby("model")[["token_total", "token_cost"]].sum().T.round(2)
token_cost_summary.index = ["Total Tokens Used", "Total Cost ($)"]

# 4. Combine all into a multi-section table
combined_table = pd.concat([
    score_table,
    pd.DataFrame([""] * score_table.shape[1], index=score_table.columns).T,  # empty row
    overall_avg,
    pd.DataFrame([""] * score_table.shape[1], index=score_table.columns).T,  # empty row
    token_cost_summary
])

# 5. Display
print("=== Benchmark Summary ===")
print(combined_table)

=== Benchmark Summary ===
model                   4o-base 4o-textgrad   o1-base
chitchat/out_of_scope      0.75        0.86      0.75
clarification              0.65        0.64      0.75
conversation_end          0.825       0.875      0.75
multi_intent               0.45        0.75      0.82
safety                     0.77        0.73      0.65
straight_admin              0.7        0.68       0.7
straight_education          0.7        0.85      0.75
straight_patient           0.49         0.9      0.25
straight_scheduling        0.75        0.85      0.75
0                                                    
Overall Average Score    0.6655      0.7869     0.681
0                                                    
Total Tokens Used         11619       20752     25750
Total Cost ($)         0.037358    0.064113  1.073895


In [56]:
print(combined_table.to_markdown())


|                       | 4o-base   | 4o-textgrad   | o1-base   |
|:----------------------|:----------|:--------------|:----------|
| chitchat/out_of_scope | 0.75      | 0.86          | 0.75      |
| clarification         | 0.65      | 0.64          | 0.75      |
| conversation_end      | 0.825     | 0.875         | 0.75      |
| multi_intent          | 0.45      | 0.75          | 0.82      |
| safety                | 0.77      | 0.73          | 0.65      |
| straight_admin        | 0.7       | 0.68          | 0.7       |
| straight_education    | 0.7       | 0.85          | 0.75      |
| straight_patient      | 0.49      | 0.9           | 0.25      |
| straight_scheduling   | 0.75      | 0.85          | 0.75      |
| 0                     |           |               |           |
| Overall Average Score | 0.6655    | 0.7869        | 0.681     |
| 0                     |           |               |           |
| Total Tokens Used     | 11619     | 20752         | 25750     |
| Total Co

In [57]:
md = combined_table.to_markdown(tablefmt="pipe")
print(md)

|                       | 4o-base   | 4o-textgrad   | o1-base   |
|:----------------------|:----------|:--------------|:----------|
| chitchat/out_of_scope | 0.75      | 0.86          | 0.75      |
| clarification         | 0.65      | 0.64          | 0.75      |
| conversation_end      | 0.825     | 0.875         | 0.75      |
| multi_intent          | 0.45      | 0.75          | 0.82      |
| safety                | 0.77      | 0.73          | 0.65      |
| straight_admin        | 0.7       | 0.68          | 0.7       |
| straight_education    | 0.7       | 0.85          | 0.75      |
| straight_patient      | 0.49      | 0.9           | 0.25      |
| straight_scheduling   | 0.75      | 0.85          | 0.75      |
| 0                     |           |               |           |
| Overall Average Score | 0.6655    | 0.7869        | 0.681     |
| 0                     |           |               |           |
| Total Tokens Used     | 11619     | 20752         | 25750     |
| Total Co

In [58]:
combined_table.to_csv("frontdesk_agent_benchmark_summary.csv")
